In [1]:
from scipy import io
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
from quaternion import Quaternion
from estimate_rot import accel_calib_params, accel_calibration, gyro_calib_params, gyro_calibration

In [2]:
data_num = 1
imu = io.loadmat('imu/imuRaw'+str(data_num)+'.mat')
accel = imu['vals'][0:3,:] # order: roll, pitch, yaw
gyro = imu['vals'][3:6,:]
T = np.shape(imu['ts'])[1]
ts_imu = imu['ts'].reshape(-1,)
angle_names = ['roll', 'pitch', 'yaw']


accel_bias, accel_sensitivity = accel_calib_params(accel)
accel = accel_calibration(accel, accel_bias, accel_sensitivity)

gyro_bias, gyro_sensitivity = gyro_calib_params(gyro)
gyro = gyro_calibration(gyro, gyro_bias, gyro_sensitivity)

vicon = io.loadmat('vicon/viconRot'+str(data_num)+'.mat')
T_vicon = np.shape(vicon['ts'])[1]
ts_vicon = vicon['ts'].reshape(-1,)

In [3]:
euler_vicon = []
for t in range(T_vicon):
    vicon_r = Rotation.from_matrix(vicon['rots'][:,:,t])
    euler_vicon.append(vicon_r.as_euler("zyx")) #yaw, pitch, roll
euler_vicon = np.array(euler_vicon).T
euler_vicon[[2,0], :] = euler_vicon[[0,2], :]

# plt.figure(figsize = (10,3))
# for i in range(3):
#     plt.plot(ts_vicon, euler_vicon[i,:], label = angle_names[i])
# plt.legend()
# plt.title('Vicon Euler angles data')

In [8]:
Q = Quaternion(scalar = 1, vec = [2,3,4]) 
w0_x, w0_y, w0_z = 0, 0, 0
state = (Q.q, w0_x, w0_y, w0_z)

Q.from_axis_angle(1.2)
state


(array([0.82533561, 0.56464247, 0.56464247, 0.56464247]), 0, 0, 0)